In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from pof.component import Component
from pof.failure_mode import FailureMode
import pof.demo as demo
from pof.loader.asset_model_loader import AssetModelLoader
from pof.paths import Paths
from pof.data.asset_data import SimpleFleet

In [2]:
# Forecast years
START_YEAR = 2015
END_YEAR = 2024
CURRENT_YEAR = 2020

paths = Paths()

# Population Data
file_path = paths.input_path + os.sep
FILE_NAME = r"population_summary.csv"

sfd = SimpleFleet(file_path + FILE_NAME)
sfd.load()
sfd.calc_age_forecast(START_YEAR, END_YEAR, CURRENT_YEAR);

In [5]:
aml = AssetModelLoader(paths.demo_path + os.sep + "Asset Model - Pole - Timber.xlsx")
comp_data = aml.load(paths.demo_path + os.sep + "Asset Model - Pole - Timber.xlsx")
comp = Component.from_dict(comp_data["pole"])
comp.fleet_data = sfd

In [8]:
t_end = 1000
n_iterations = 100

In [43]:
# Turn failure modes on & off

failure_modes_on = ["termites", "fungal decay | internal"]
tasks_off = ['pole_saver_rod']

for fm in comp.fm.values():
    if fm.name not in failure_modes_on:
        fm.active = False
    else:
        fm.active = True

    for task in fm.tasks.values():
        if task.name in tasks_off:
            task.active = False
        else:
            task.active = True

In [44]:
comp.fm['fungal decay | internal'].tasks['inspection_groundline'].p_effective = 1

In [45]:
comp.mc_timeline(t_end=t_end, n_iterations=n_iterations)

100%|██████████| 100/100 [00:06<00:00, 15.32it/s]


In [12]:
comp.expected_risk_cost_df()

,failure_mode,task,active,time,quantity,cost,cost_cumulative,cost_annual,quantity_cumulative,quantity_annual
28,termites,risk,True,0,0,0,0,0,0,0
29,fungal decay | external,risk,False,0,0,0,0,0,0,0
30,fungal decay | internal,risk,True,0,0,0,0,0,0,0
31,lightning,risk,False,0,0,0,0,0,0,0
32,weathering,risk,False,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
45,lightning,inspection_protection,False,181,0,0,0,0,0,0
47,fire_damage,inspection_protection,False,181,0,0,0,0,0,0
48,impact,inspection_protection,False,181,0,0,0,0,0,0
37,fungal decay | internal,pole_saver_rod,False,181,0,0,0,0,0,0


In [13]:
fms_to_check = ["termites", "fungal decay | internal"]
tasks_to_check = ["conditional_failure", "functional_failure"]

In [14]:
dict_index = dict()

for fm_name, fm in comp.fm.items():
    if fm_name in fms_to_check:
        for index, timeline in fm._timelines.items():
            for task_name in tasks_to_check:
               if 0 in timeline[task_name]: 
                   dict_index[index] = {"fm":fm_name, "task":task_name}

dict_index

{1: {'fm': 'termites', 'task': 'conditional_failure'},
 3: {'fm': 'termites', 'task': 'conditional_failure'},
 7: {'fm': 'termites', 'task': 'conditional_failure'},
 8: {'fm': 'termites', 'task': 'conditional_failure'},
 9: {'fm': 'termites', 'task': 'conditional_failure'},
 14: {'fm': 'termites', 'task': 'functional_failure'},
 16: {'fm': 'termites', 'task': 'functional_failure'},
 17: {'fm': 'termites', 'task': 'functional_failure'},
 21: {'fm': 'termites', 'task': 'conditional_failure'},
 22: {'fm': 'termites', 'task': 'conditional_failure'},
 23: {'fm': 'termites', 'task': 'conditional_failure'},
 25: {'fm': 'termites', 'task': 'conditional_failure'},
 26: {'fm': 'termites', 'task': 'conditional_failure'},
 27: {'fm': 'termites', 'task': 'conditional_failure'},
 29: {'fm': 'termites', 'task': 'conditional_failure'},
 31: {'fm': 'fungal decay | internal', 'task': 'conditional_failure'},
 35: {'fm': 'termites', 'task': 'conditional_failure'},
 36: {'fm': 'termites', 'task': 'function

In [ ]:
fm = comp.fm['fungal decay | internal']
timeline = fm.timeline

fm.tasks['conditional_failure'].sim_timeline(
                        t_start=0, t_end=160, timeline=timeline, indicators = fm.indicators
                    )

In [18]:
comp.fm['fungal decay | internal']._timelines[0]

{'time': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]),
 'initiation': array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, Fals

In [29]:
comp.fm["termites"].indicators["safety_factor"]._timelines = None;

comp.fm["termites"].indicators["safety_factor"].__dict__;


In [64]:
comp.reset()
comp.sim_timeline(t_end)

In [65]:
comp.fm['fungal decay | internal'].timeline
#comp.fm['termites'].timeline

{'time': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111]),
 'initiation': array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, Fal

In [42]:
comp.indicator['wall_thickness']._timelines[25]['termites']

array([125.        , 125.        , 125.        , ...,  19.88794809,
        19.88794809,  19.88794809])

In [58]:
comp.fm['fungal decay | internal'].states

{'initiation': False, 'detection': True, 'failure': False}